In [ ]:
import folium
import pandas as pd
import os
import colorsys
from branca.colormap import linear

# 파일 경로와 범위
base_path = "/content/drive/MyDrive/여의도_불꽃축제_데이터/인구종합/"
file_template = "2024-10-05_{:02d}_인구종합.csv"
output_path = "/content/drive/MyDrive/여의도_불꽃축제_데이터/지도결과/2024/"

# 결과 저장 경로 확인 및 생성
os.makedirs(output_path, exist_ok=True)

# 인구수 구간에 대한 기준 설정
population_limits = {
    '여유': 2500,
    '관심': 6000,
    '주의': 8000,
    '경고': 10000
}

# 컬러 맵 정의 (붉은색 계열, 명도 조정)
def get_color(population, limits):
    if population <= limits['여유']:
        lightness = 0.9  # 명도 25
    elif population <= limits['관심']:
        lightness = 0.7  # 명도 50
    elif population <= limits['주의']:
        lightness = 0.5  # 명도 75
    else:
        lightness = 0.3  # 명도 100

    # Hue를 0으로 고정하여 붉은색 계열 생성
    saturation = 0.8
    r, g, b = colorsys.hls_to_rgb(0, lightness, saturation)  # 붉은색 계열
    return f"rgb({int(r * 255)}, {int(g * 255)}, {int(b * 255)})"  # RGB로 변환 후 반환

# 전체 최대 인구수 계산을 위해 초기화
all_population = []

# 지도 생성 및 파일 저장
for hour in range(24):  # 00~23까지 반복
    file_path = base_path + file_template.format(hour)

    try:
        # 파일 읽기
        df = pd.read_csv(file_path)

        # 해당 시간대의 인구수를 all_population 리스트에 저장
        all_population.extend(df['POPL_CNT'])

    except FileNotFoundError:
        print(f"File not found: {file_path}")
        continue

# 고정된 중심 좌표 설정 (gif를 제작하기 위해)
fixed_center_lat = 37.526194
fixed_center_lon = 126.916575

# 각 시간대별로 지도 생성
for hour in range(24):  # 00~23까지 반복
    file_path = base_path + file_template.format(hour)

    try:
        # 파일 읽기
        df = pd.read_csv(file_path)

        # 지도 초기화 (고정된 중심 좌표 사용)
        map_folium = folium.Map(location=[fixed_center_lat, fixed_center_lon], zoom_start=15, width='100%', height='80%')

        # 각 좌표에 사각형 그리기
        grid_size = 0.0005  # 약 50m 간격
        for _, row in df.iterrows():
            folium.Rectangle(
                bounds=[
                    [row['Y_COORD'] - grid_size / 2, row['X_COORD'] - grid_size / 2],
                    [row['Y_COORD'] + grid_size / 2, row['X_COORD'] + grid_size / 2]
                ],
                color=None,
                fill=True,
                fill_color=get_color(row['POPL_CNT'], population_limits),  # 인구수에 따라 색상 조정
                fill_opacity=0.8
            ).add_to(map_folium)

        # 컬러맵 범례 추가 (Red to Dark Red)
        colormap = linear.Reds_09.scale(0, population_limits['경고'])  # 경고 기준에 맞춰 범례 설정
        colormap.caption = 'Population Density'
        map_folium.add_child(colormap)

        # 지도 저장
        map_file_path = output_path + f"2024-10-05_{hour:02d}_grid_map.html"
        map_folium.save(map_file_path)
        print(f"Saved map for {hour}: {map_file_path}")

    except FileNotFoundError:
        print(f"File not found: {file_path}")
        continue